In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nyt-comments/CommentsFeb2018.csv
/kaggle/input/nyt-comments/ArticlesFeb2017.csv
/kaggle/input/nyt-comments/CommentsApril2018.csv
/kaggle/input/nyt-comments/ArticlesJan2017.csv
/kaggle/input/nyt-comments/ArticlesMay2017.csv
/kaggle/input/nyt-comments/CommentsJan2017.csv
/kaggle/input/nyt-comments/CommentsMarch2017.csv
/kaggle/input/nyt-comments/CommentsMay2017.csv
/kaggle/input/nyt-comments/CommentsMarch2018.csv
/kaggle/input/nyt-comments/CommentsApril2017.csv
/kaggle/input/nyt-comments/ArticlesMarch2017.csv
/kaggle/input/nyt-comments/ArticlesApril2017.csv
/kaggle/input/nyt-comments/CommentsFeb2017.csv
/kaggle/input/nyt-comments/ArticlesJan2018.csv
/kaggle/input/nyt-comments/ArticlesFeb2018.csv
/kaggle/input/nyt-comments/ArticlesMarch2018.csv
/kaggle/input/nyt-comments/CommentsJan2018.csv
/kaggle/input/nyt-comments/ArticlesApril2018.csv


In [2]:
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import torch

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Dense, MultiHeadAttention, LayerNormalization, Dropout, Bidirectional, Flatten, Layer, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model

from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

2024-07-07 04:50:37.318624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 04:50:37.318724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 04:50:37.431656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
working_dir = '../input/nyt-comments/'

for filename in os.listdir(working_dir):
    if 'Articles' in filename:
        df = pd.read_csv(working_dir + filename)
df

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,5ae82c93068401528a2ab969,1004,By CLAIRE CAIN MILLER,article,This Common Question Reinforces the Gender Pay...,"['Discrimination', 'Wages and Salaries', 'Labo...",68,Upshot,3,2018-05-01 09:00:01,Unknown,Several states and cities have ordered employe...,The New York Times,News,https://www.nytimes.com/2018/05/01/upshot/how-...
1320,5ae82c95068401528a2ab96b,1043,By TRACY J. GATES,article,"Anna, Llama and Me","['Friendship', 'Dewdney, Anna', 'Writing and W...",65,Well,0,2018-05-01 09:00:02,Family,"The beginning, middle and end of a picture boo...",The New York Times,News,https://www.nytimes.com/2018/05/01/well/family...
1321,5ae82c9d068401528a2ab96d,659,Interview by AUDIE CORNISH,article,Gen. Michael Hayden Has One Regret: Russia,"['Classified Information and State Secrets', '...",66,Magazine,70,2018-05-01 09:00:06,Unknown,"The former N.S.A. and C.I.A. chief on Trump, S...",The New York Times,News,https://www.nytimes.com/2018/05/01/magazine/ge...
1322,5ae82c9f068401528a2ab96f,1155,By JASON ROBERT BROWN,article,There Is Nothin’ Like a Tune,"['Books and Literature', 'Purdum, Todd S', 'Th...",68,BookReview,17,2018-05-01 09:00:07,Book Review,"In “Something Wonderful,” Todd S. Purdum analy...",The New York Times,Review,https://www.nytimes.com/2018/05/01/books/revie...


In [4]:
print(df.head())

                  articleID  articleWordCount  \
0  5adf6684068401528a2aa69b               781   
1  5adf653f068401528a2aa697               656   
2  5adf4626068401528a2aa628              2427   
3  5adf40d2068401528a2aa619               626   
4  5adf3d64068401528a2aa60f               815   

                                      byline documentType  \
0                             By JOHN BRANCH      article   
1                           By LISA FRIEDMAN      article   
2                              By PETE WELLS      article   
3  By JULIE HIRSCHFELD DAVIS and PETER BAKER      article   
4             By IAN AUSTEN and DAN BILEFSKY      article   

                                            headline  \
0  Former N.F.L. Cheerleaders’ Settlement Offer: ...   
1  E.P.A. to Unveil a New Rule. Its Effect: Less ...   
2                            The New Noma, Explained   
3                                            Unknown   
4                                            Unknown   

 

In [5]:
working_dir = '../input/nyt-comments/'
headlines = []

for filename in os.listdir(working_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(working_dir + filename)
        headlines.extend(list(article_df.headline.values))

headlines = [x for x in headlines if x!="Unknown"]

In [6]:
len(headlines)
sample = headlines[:10]
sample

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree']

In [7]:
import string
def cleaning(data):
    cleaned = "".join(x for x in data if x not in string.punctuation).lower()
    cleaned = cleaned.encode("utf8").decode("ascii", "ignore")
    return cleaned

cleaned_l = [cleaning(x) for x in headlines]
cleaned_l[:10]

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean',
 'trail activity',
 'super bowl',
 'trumps mexican shakedown',
 'pences presidential pet',
 'fruit of a poison tree']

In [8]:
print(type(cleaned_l))
cleaned_l[:10]

<class 'list'>


['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean',
 'trail activity',
 'super bowl',
 'trumps mexican shakedown',
 'pences presidential pet',
 'fruit of a poison tree']

In [9]:
corpus = cleaned_l
tk = Tokenizer()
tk.fit_on_texts(corpus)
tk.word_index

{'the': 1,
 'a': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'for': 6,
 'and': 7,
 'on': 8,
 'is': 9,
 'trump': 10,
 'with': 11,
 'new': 12,
 'at': 13,
 'how': 14,
 'what': 15,
 'you': 16,
 'an': 17,
 'from': 18,
 'as': 19,
 'it': 20,
 'trumps': 21,
 'its': 22,
 'your': 23,
 'are': 24,
 'be': 25,
 'not': 26,
 'us': 27,
 'season': 28,
 'that': 29,
 'by': 30,
 'about': 31,
 'but': 32,
 'can': 33,
 'episode': 34,
 'do': 35,
 'this': 36,
 'when': 37,
 'york': 38,
 'up': 39,
 'over': 40,
 'no': 41,
 'why': 42,
 'i': 43,
 'out': 44,
 'more': 45,
 'my': 46,
 'after': 47,
 'will': 48,
 'may': 49,
 'or': 50,
 'we': 51,
 'war': 52,
 'who': 53,
 'his': 54,
 'health': 55,
 'teaching': 56,
 'questions': 57,
 'now': 58,
 'president': 59,
 'was': 60,
 'one': 61,
 'house': 62,
 'today': 63,
 'get': 64,
 'should': 65,
 'have': 66,
 'into': 67,
 'all': 68,
 'home': 69,
 'dont': 70,
 'life': 71,
 'our': 72,
 'has': 73,
 'plan': 74,
 'good': 75,
 'first': 76,
 'gop': 77,
 '1': 78,
 'says': 79,
 'like': 80,
 'white'

In [10]:
# Creation of N-Grams
tokenizer = Tokenizer()

#takes a list of text strings and processes it to generate n-gram sequences.
def get_sequence(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
# converting the text to a list of integers based on the tokenizer's word index    
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences,total_words
input_sequences ,total_words_in_corp = get_sequence(corpus)

In [11]:
print(f"Input Sequence Example: \n{input_sequences[:8]}\n")
print(f"Total Number of Words in Corpus: {total_words_in_corp}")

Input Sequence Example: 
[[915, 148], [915, 148, 212], [915, 148, 212, 73], [915, 148, 212, 73, 475], [915, 148, 212, 73, 475, 202], [915, 148, 212, 73, 475, 202, 68], [915, 148, 212, 73, 475, 202, 68, 28], [915, 148, 212, 73, 475, 202, 68, 28, 182]]

Total Number of Words in Corpus: 11265


In [12]:
print(len(input_sequences))
print(total_words_in_corp)

51770
11265


In [13]:
import keras.utils as ku
def generate_padded_seq(input_sequences):
    max_seq_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences,maxlen = max_seq_len,padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label,num_classes = total_words_in_corp)
    return predictors, label, max_seq_len

In [14]:
predictors, label, max_sequence_len = generate_padded_seq(input_sequences)
predictors, label, max_sequence_len

(array([[    0,     0,     0, ...,     0,     0,   915],
        [    0,     0,     0, ...,     0,   915,   148],
        [    0,     0,     0, ...,   915,   148,   212],
        ...,
        [    0,     0,     0, ...,   242,     9, 11264],
        [    0,     0,     0, ...,     9, 11264,    80],
        [    0,     0,     0, ..., 11264,    80,     2]], dtype=int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 24)

In [15]:
display(input_sequences[:10])

[[915, 148],
 [915, 148, 212],
 [915, 148, 212, 73],
 [915, 148, 212, 73, 475],
 [915, 148, 212, 73, 475, 202],
 [915, 148, 212, 73, 475, 202, 68],
 [915, 148, 212, 73, 475, 202, 68, 28],
 [915, 148, 212, 73, 475, 202, 68, 28, 182],
 [436, 3366],
 [436, 3366, 5166]]

In [16]:
max_len = max([len(x) for x in input_sequences])
print(max_len)

24


In [17]:
padded_input_seq = pad_sequences(input_sequences,maxlen=max_len,padding='pre')

X = padded_input_seq[:,:-1]
y = padded_input_seq[:,-1]

In [18]:
y = to_categorical(y)

In [19]:
X,y

(array([[    0,     0,     0, ...,     0,     0,   915],
        [    0,     0,     0, ...,     0,   915,   148],
        [    0,     0,     0, ...,   915,   148,   212],
        ...,
        [    0,     0,     0, ...,   242,     9, 11264],
        [    0,     0,     0, ...,     9, 11264,    80],
        [    0,     0,     0, ..., 11264,    80,     2]], dtype=int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [20]:
X.shape,y.shape

((51770, 23), (51770, 11265))

In [21]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [22]:
voc = len(tokenizer.word_index) + 1

In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, Dense, LayerNormalization, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
inputs = Input(shape = (max_len-1,))
embedding_layer = Embedding(voc,200)(inputs)
transformer_block = TransformerBlock(embed_dim=200,num_heads=4,ff_dim=128)
x = transformer_block(embedding_layer,training=True)
x = LayerNormalization(epsilon=1e-6)(x)
x = Dropout(0.2)(x)
x = Dense(512)(x)
x = Dropout(0.2)(x)
x = Flatten()(x)

outputs = Dense(voc, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 23)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 23, 200)        │     2,253,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 23, 200)        │       694,928 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 23, 200)        │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 23, 200)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 23, 512)        │       102,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 23, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 11776)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 11265)          │   132,667,905 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,719,145 (517.73 MB)

 Trainable params: 135,719,145 (517.73 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
rlrong = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    mode = 'min',
    min_lr = 1e-5,
    patience = 2,
    verbose = 1
)
estop = EarlyStopping(
    monitor = 'val_loss',
    mode = 'min',
    patience = 30,
    verbose = 1,
    restore_best_weights = True
)

In [26]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen = max_sequence_len - 1,padding = "pre")
        predicted = model.predict(token_list,verbose = 0)
        predicted = np.argmax(predicted, axis=-1)

        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

In [27]:
history = model.fit(X, y, epochs = 10, verbose = 1, validation_split=0.2, callbacks = [rlrong,estop])

Epoch 1/10
   2/1295 ━━━━━━━━━━━━━━━━━━━━ 1:06 51ms/step - accuracy: 0.0078 - loss: 9.9064      

I0000 00:00:1720327875.323435      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720327875.347098      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1295/1295 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0172 - loss: 9.0271

W0000 00:00:1720327939.485315      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720327941.724667      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720327944.407507      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1295/1295 ━━━━━━━━━━━━━━━━━━━━ 84s 54ms/step - accuracy: 0.0172 - loss: 9.0268 - val_accuracy: 0.0676 - val_loss: 7.7026 - learning_rate: 0.0010
Epoch 2/10
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 58s 44ms/step - accuracy: 0.1363 - loss: 6.4416 - val_accuracy: 0.0975 - val_loss: 7.6622 - learning_rate: 0.0010
Epoch 3/10
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 56s 43ms/step - accuracy: 0.5399 - loss: 2.5709 - val_accuracy: 0.0794 - val_loss: 9.0243 - learning_rate: 0.0010
Epoch 4/10
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7076 - loss: 1.5409
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 56s 43ms/step - accuracy: 0.7076 - loss: 1.5410 - val_accuracy: 0.0844 - val_loss: 10.8872 - learning_rate: 0.0010
Epoch 5/10
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 56s 43ms/step - accuracy: 0.7917 - loss: 1.0282 - val_accuracy: 0.0829 - val_loss: 11.4631 - learning_rate: 2.0000e-04
Epoch 6/10
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8

In [28]:
print(generate_text("Python",20,model,max_sequence_len))

Python And The New York Today The Makeshift Law Office Closes At Kennedy Office Or Just Defiance And Rejection Again Or


In [29]:
print(generate_text("Politics in usa",20,model,max_sequence_len))

Politics In Usa And The Real Dead Season 6 Episode 9 The Neverending Of Trump Dying Obvious The Adal World Of The Real


In [30]:
print(generate_text("India",20,model,max_sequence_len))

India In A New York Today A Makeshift About The Subway Of Kennedy Schools To The New York Today Washington And
